In [2]:
import pyreadr
import pandas as pd
import os
from RaceID import SCseq
import varID_functions
from scipy.sparse import csr_matrix
import numpy as np
import RaceID_utils
import re
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [3]:
input_data = './input_data/'
rds_files = ['wt.rds', 'EPO.rds', 'tf.rds']
data_object = []
for file in rds_files:
    file_path = os.path.join(input_data, file)
    result = pyreadr.read_r(file_path)
    data_object.append(result[None])

In [4]:
wt = data_object[0]

In [5]:
sN = SCseq(wt)

In [6]:

# Apply the regular expression pattern
pattern = "^(mt|Rp(l|s)|Gm\\d)"
CGenes = [gene_name for idx, gene_name in enumerate(sN.expdata.index) if re.match(pattern, gene_name)]

sN.filterdata(mintotal=1000, CGenes=CGenes)

In [7]:
sN.expdata.shape

(28205, 5432)

```R
> dim(sN@expdata)
[1] 28205  5432
```

In [8]:
sN.ndata.shape

(28205, 4742)

```R
> dim(sN@ndata)
[1] 28205  4742
```

In [9]:
len(sN.genes)

3439

```R
> length(sN@genes)
[1] 3439
```

### fitted model params

In [10]:
print(sN.background['vfit'].params, '\n',sN.background['vfit'].bse)

Intercept     1.167001
ml            1.462929
I(ml ** 2)    0.144933
dtype: float64 
 Intercept     0.011766
ml            0.008179
I(ml ** 2)    0.003251
dtype: float64


R output
```R
> summary(sN@background$vfit)

Call:
lm(formula = vl ~ ml + I(ml^2))

Residuals:
    Min      1Q  Median      3Q     Max 
-2.0597 -0.2316 -0.1075  0.0916  6.8109 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 1.167305   0.011766   99.21   <2e-16 ***
ml          1.462929   0.008179  178.86   <2e-16 ***
I(ml^2)     0.144933   0.003251   44.59   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4794 on 3436 degrees of freedom
Multiple R-squared:  0.9092,	Adjusted R-squared:  0.9091 
F-statistic: 1.72e+04 on 2 and 3436 DF,  p-value: < 2.2e-16
```

### cluster features

In [11]:
len(sN.cluster['features'])

1031

R output
```R
> length(sN@cluster$features)
[1] 1031
```


expData  <- as.matrix(getExpData(sN))
res      <- pruneKnn(expData,no_cores=5,knn=100)
cl    <- graphCluster(res,pvalue=0.01,use.leiden=TRUE,leiden.resolution=1)
probs <- transitionProbs(res,cl)

sN <- updateSC(sN,res=res,cl=cl)
sN <- comptsne(sN)
sN <- compumap(sN)
plotmap(sN, um=TRUE)

In [12]:
expData = sN.getExpData()

In [13]:
expData

,bBM1,bBM2,bBM3,bBM4,bBM5,bBM6,bBM7,bBM8,bBM9,bBM10,...,bBM5241,bBM5253,bBM5266,bBM5272,bBM5282,bBM5290,bBM5297,bBM5304,bBM5312,bBM5315
X0610007P14Rik,6,0,3,2,2,1,1,0,0,5,...,0,0,0,0,0,0,0,0,0,0
X0610010K14Rik,2,4,3,2,3,1,2,6,1,2,...,0,0,0,0,0,0,0,0,0,0
X1110001J03Rik,9,5,3,1,2,2,3,8,2,4,...,1,0,1,0,1,0,0,0,0,0
X1110004F10Rik,6,4,4,3,5,0,5,2,0,3,...,0,0,0,1,0,0,0,0,0,0
X1110007C09Rik,0,0,0,1,0,0,3,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zranb1,3,10,2,1,0,4,2,0,2,0,...,0,0,0,1,0,0,0,0,0,0
Zranb2,7,1,9,0,6,5,4,0,2,0,...,0,0,0,1,0,0,1,0,0,0
Zwint,5,4,4,0,5,2,9,6,7,0,...,0,0,0,0,0,0,3,0,0,0
Zyx,2,2,1,1,1,2,0,1,0,0,...,3,1,0,0,1,2,0,0,0,0


In [14]:
varID_functions.getFormula0()

'x ~ 0'

In [17]:
k = np.log(expData.sum(axis=0))
k

bBM1       9.915910
bBM2       9.726512
bBM3       9.417436
bBM4       9.477386
bBM5       9.737020
             ...   
bBM5290    6.466145
bBM5297    6.517671
bBM5304    6.428105
bBM5312    6.473891
bBM5315    6.478510
Length: 4742, dtype: float64

In [18]:
varID_functions.getRegData(k)

,beta
bBM1,9.915910
bBM2,9.726512
bBM3,9.417436
bBM4,9.477386
bBM5,9.737020
...,...
bBM5290,6.466145
bBM5297,6.517671
bBM5304,6.428105
bBM5312,6.473891


In [13]:
varID_functions.pruneKnn(expData)

TypeError: '>' not supported between instances of 'int' and 'NoneType'

In [ ]:
expData.index

Index(['X0610007P14Rik', 'X0610010K14Rik', 'X1110001J03Rik', 'X1110004F10Rik',
       'X1110007C09Rik', 'X1110008F13Rik', 'X1110059G10Rik', 'X1200014J11Rik',
       'X1500012F01Rik', 'X1600014C10Rik',
       ...
       'Zmynd11', 'Zmynd19', 'Zmynd8', 'Znrf1', 'Znrf2', 'Zranb1', 'Zranb2',
       'Zwint', 'Zyx', 'l7Rn6'],
      dtype='object', length=3439)

In [ ]:
expData.loc[expData.index]

,bBM1,bBM2,bBM3,bBM4,bBM5,bBM6,bBM7,bBM8,bBM9,bBM10,...,bBM5241,bBM5253,bBM5266,bBM5272,bBM5282,bBM5290,bBM5297,bBM5304,bBM5312,bBM5315
X0610007P14Rik,6,0,3,2,2,1,1,0,0,5,...,0,0,0,0,0,0,0,0,0,0
X0610010K14Rik,2,4,3,2,3,1,2,6,1,2,...,0,0,0,0,0,0,0,0,0,0
X1110001J03Rik,9,5,3,1,2,2,3,8,2,4,...,1,0,1,0,1,0,0,0,0,0
X1110004F10Rik,6,4,4,3,5,0,5,2,0,3,...,0,0,0,1,0,0,0,0,0,0
X1110007C09Rik,0,0,0,1,0,0,3,0,0,1,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zranb1,3,10,2,1,0,4,2,0,2,0,...,0,0,0,1,0,0,0,0,0,0
Zranb2,7,1,9,0,6,5,4,0,2,0,...,0,0,0,1,0,0,1,0,0,0
Zwint,5,4,4,0,5,2,9,6,7,0,...,0,0,0,0,0,0,3,0,0,0
Zyx,2,2,1,1,1,2,0,1,0,0,...,3,1,0,0,1,2,0,0,0,0


In [ ]:
import multiprocessing

multiprocessing.cpu_count()

64

In [ ]:
max(1,2,3)

3

In [ ]:
import multiprocessing

# Function to detect number of CPU cores
def detect_cores():
    return multiprocessing.cpu_count()

# Assuming no_cores is defined somewhere earlier, e.g., no_cores = None
no_cores = 13

# Python equivalent logic
if no_cores is None:
    no_cores = max(1, detect_cores() - 2)
no_cores = min(no_cores, detect_cores())

print(f'Number of cores to use: {no_cores}')

Number of cores to use: 13


In [ ]:
np.log(expData.sum(axis=0))

bBM1       9.915910
bBM2       9.726512
bBM3       9.417436
bBM4       9.477386
bBM5       9.737020
             ...   
bBM5290    6.466145
bBM5297    6.517671
bBM5304    6.428105
bBM5312    6.473891
bBM5315    6.478510
Length: 4742, dtype: float64